## Ejercicio 3

**Diseñe razonadamente el entorno de un ejemplo realista para el intercambio de una clave según el sistema de Diffie-Hellman y proceda al intercambio de una de ellas. Para llevar a cabo este ejemplo puede usar sagemath y openssl, pero el ejemplo que construya debe ser distinto en los datos a cualquiera que figure en los apuntes (p.e. Ejemplo 5.5.3 de la pág. 122).**

En una conexión SSH se establecen dos etapas: La primera es acordar una clave de cifrado para proteger la comunicación futura y la segunda es autenticar al usuario y comprobar si se le debe dar acceso al servidor. El esquema Diffie-Hellman es utilizado en la primera de las fases, donde ambas partes negocian una clave de sesión compartida, pero secreta. Al utilizar Diffie-Hellman, cada parte puede combinar sus propios datos privados con datos públicos del otro sistema para así llegar a dicha clave compartida. 

Esta clave de sesión se utiliza para cifrar toda la sesión. No combiene confundir las claves privadas utilizadas en este paso con las claves SSH utilizadas para autenticar un cliente en el servidor, ya que son conceptos distintos y están completamente separados.

El procedimiento seguido es el siguiente. Por simplicidad llamaremos $A$ y $B$ a las dos partes.

Utilizamos `openssl` para generar un primo seguro suficientemente grande, por ejemplo, de 64 bits:

```bash
$ openssl prime -generate -safe -bits 64
17885555072688303479
```

A partir de este número primo que llamaremos $n$, elegimos $g$, otro número primo que es además un elemento primitivo de $GF(n)$:

In [ ]:
n = 17885555072688303479
g = GF(n).primitive_element()

Al ser $g$ un elemento primitivo de $GF(n)$, tenemos asegurado que $1<g<n$. La pareja $(n,g)$ no tiene por qué ser secreta, de hecho puede ser compartida por más usuarios.

In [ ]:
print("(n,g)=(%d,%d)" % (n,g))

### A elige su clave secreta y calcula X
A continuación A elige aleatoriamente un número elevado $x$ y calcula el número $X=g^x \mod n$, el cual será enviado a B.

In [ ]:
# A
x = randint(0,2**20)
X = g**x % n
print("La clave secreta de A es x = %d" % x)
print("A le enviará a B el número X = %d" % X)

### B elige su clave secreta y calcula Y
Del mismo modo, B elige aleatoriamente un número elevado $y$ y calcula el número $Y=g^y \mod n$, el cual será enviado a A.

In [ ]:
# B
y = randint(0,2**20)
Y = g**y % n
print("La clave secreta de B es y = %d" % y)
print("B le enviará a A el número Y = %d" % Y)

### Intercambio
A y B se intercambian $X$ e $Y$, manteniendo secretos los exponentes $x$ e $y$. A recibe $Y$ y calcula $k_1$:

In [ ]:
# A
k1 = Y**x % n
print("Clave calculada por A: %d" % k1)

B recibe $X$ y calcula $k_2$

In [ ]:
# B
k2 = X**y % n
print("Clave calculada por B: %d" % k2)

Comprobamos que en efecto $k_1$ y $k_2$ coinciden, siendo calculadas independientemente y sin revelar las claves secretas $x$ e $y$. Este número $k_1=k_2$ es la clave secreta compartida entre A y B, que estas utilizarán posteriormente para cifrar la comunicación en SSH.

In [ ]:
k1 == k2

Este proceso permite a cada parte participar igualmente en la generación del secreto compartido, lo que no permite que un extremo controle el secreto. También cumple la tarea de generar un secreto compartido idéntico sin tener que enviar esa información a través de canales inseguros. La clave generada es simétrica, lo que significa que la misma clave utilizada para cifrar un mensaje se puede utilizar para descifrarlo en el otro lado. El propósito de esto es envolver toda la comunicación adicional en un túnel encriptado que no pueda ser descifrado por personas externas.